In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
spacex_file = "spacex_launch_data.csv"
teslatwo_file="TSLA _to_29th.csv"

In [3]:
spacex_df = pd.read_csv(spacex_file)
teslatwo_df=pd.read_csv(teslatwo_file)

In [4]:
spacex_df.head( )

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Mission Outcome,Landing Outcome
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,NaN,LEO,SpaceX,Success,Failure (parachute)
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",NaN,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525,LEO (ISS),NASA (COTS),Success,No attempt
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [5]:
spacex_cols = ["Flight Number", "Date","Launch Site", "Payload", "Customer", "Mission Outcome", "Landing Outcome"]
spacex_transform = spacex_df[spacex_cols].copy()

spacex_transform = spacex_transform.rename(columns={"Flight Number":"id",
                                                    "Date":"real_date",
                                                    "Launch Site": "launch_site",
                                                    "Payload":"payload",
                                                    "Customer":"customer",
                                                    "Mission Outcome":"mission_outcome",
                                                    "Landing Outcome":"landing_outcome"})

spacex_transform.drop_duplicates("id", inplace=True)
spacex_transform.set_index("id", inplace=True)

spacex_transform.head()


,real_date,launch_site,payload,customer,mission_outcome,landing_outcome
id,,,,,,
1,2010-06-04,CCAFS LC-40,Dragon Spacecraft Qualification Unit,SpaceX,Success,Failure (parachute)
2,2010-12-08,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",NASA (COTS) NRO,Success,Failure (parachute)
3,2012-05-22,CCAFS LC-40,Dragon demo flight C2+,NASA (COTS),Success,No attempt
4,2012-10-08,CCAFS LC-40,SpaceX CRS-1,NASA (CRS),Success,No attempt
5,2013-03-01,CCAFS LC-40,SpaceX CRS-2,NASA (CRS),Success,No attempt


In [6]:
teslatwo_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-06-29,19.000000,25.00,17.540001,23.889999,23.889999,18766300
1,2010-06-30,25.790001,30.42,23.299999,23.830000,23.830000,17187100
2,2010-07-01,25.000000,25.92,20.270000,21.959999,21.959999,8218800
3,2010-07-02,23.000000,23.10,18.709999,19.200001,19.200001,5139800
4,2010-07-06,20.000000,20.00,15.830000,16.110001,16.110001,6866900


In [7]:
teslatwo_cols = ["Date","Open","High","Low","Close","Adj Close","Volume"]
tesla_transform = teslatwo_df[teslatwo_cols].copy()

tesla_transform = tesla_transform.rename(columns={"Date":"real_date",
                                                  "Open":"open_price",
                                                  "High":"high_price",
                                                  "Low":"low_price",
                                                  "Close":"closing_price",
                                                  "Adj Close":"adj_close",
                                                  "Volume":"volume"})

tesla_transform.drop_duplicates("real_date", inplace=True)
tesla_transform.set_index("real_date")
tesla_transform.head()

,real_date,open_price,high_price,low_price,closing_price,adj_close,volume
0,2010-06-29,19.000000,25.00,17.540001,23.889999,23.889999,18766300
1,2010-06-30,25.790001,30.42,23.299999,23.830000,23.830000,17187100
2,2010-07-01,25.000000,25.92,20.270000,21.959999,21.959999,8218800
3,2010-07-02,23.000000,23.10,18.709999,19.200001,19.200001,5139800
4,2010-07-06,20.000000,20.00,15.830000,16.110001,16.110001,6866900


In [8]:
#tesla_transform.sort_values(by='real_date', ascending=False)

In [9]:
tesla_transform["range"] = tesla_transform["high_price"].subtract(tesla_transform["low_price"], fill_value=0)

In [10]:
tesla_transform.head()

,real_date,open_price,high_price,low_price,closing_price,adj_close,volume,range
0,2010-06-29,19.000000,25.00,17.540001,23.889999,23.889999,18766300,7.459999
1,2010-06-30,25.790001,30.42,23.299999,23.830000,23.830000,17187100,7.120001
2,2010-07-01,25.000000,25.92,20.270000,21.959999,21.959999,8218800,5.650000
3,2010-07-02,23.000000,23.10,18.709999,19.200001,19.200001,5139800,4.390001
4,2010-07-06,20.000000,20.00,15.830000,16.110001,16.110001,6866900,4.170000


In [11]:
min_range = tesla_transform["range"].min()
max_range = tesla_transform["range"].max()
avg_range = tesla_transform["range"].mean()

In [12]:
tesla_transform["rng_perct_close"] =(tesla_transform["range"]/tesla_transform["closing_price"])*100

In [13]:
tesla_transform.sort_values(by='real_date',ascending=True).head()

,real_date,open_price,high_price,low_price,closing_price,adj_close,volume,range,rng_perct_close
0,2010-06-29,19.000000,25.00,17.540001,23.889999,23.889999,18766300,7.459999,31.226452
1,2010-06-30,25.790001,30.42,23.299999,23.830000,23.830000,17187100,7.120001,29.878309
2,2010-07-01,25.000000,25.92,20.270000,21.959999,21.959999,8218800,5.650000,25.728599
3,2010-07-02,23.000000,23.10,18.709999,19.200001,19.200001,5139800,4.390001,22.864587
4,2010-07-06,20.000000,20.00,15.830000,16.110001,16.110001,6866900,4.170000,25.884542


In [32]:
tesla_transform.to_csv("teslanew.csv")

In [15]:
spacex_transform.head()

,real_date,launch_site,payload,customer,mission_outcome,landing_outcome
id,,,,,,
1,2010-06-04,CCAFS LC-40,Dragon Spacecraft Qualification Unit,SpaceX,Success,Failure (parachute)
2,2010-12-08,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",NASA (COTS) NRO,Success,Failure (parachute)
3,2012-05-22,CCAFS LC-40,Dragon demo flight C2+,NASA (COTS),Success,No attempt
4,2012-10-08,CCAFS LC-40,SpaceX CRS-1,NASA (CRS),Success,No attempt
5,2013-03-01,CCAFS LC-40,SpaceX CRS-2,NASA (CRS),Success,No attempt


In [33]:
spacex_transform.to_csv("spacexnew.csv")

In [17]:
tesla_transform.dropna(how="any")

,real_date,open_price,high_price,low_price,closing_price,adj_close,volume,range,rng_perct_close
0,2010-06-29,19.000000,25.000000,17.540001,23.889999,23.889999,18766300,7.459999,31.226452
1,2010-06-30,25.790001,30.420000,23.299999,23.830000,23.830000,17187100,7.120001,29.878309
2,2010-07-01,25.000000,25.920000,20.270000,21.959999,21.959999,8218800,5.650000,25.728599
3,2010-07-02,23.000000,23.100000,18.709999,19.200001,19.200001,5139800,4.390001,22.864587
4,2010-07-06,20.000000,20.000000,15.830000,16.110001,16.110001,6866900,4.170000,25.884542
5,2010-07-07,16.400000,16.629999,14.980000,15.800000,15.800000,6921700,1.649999,10.443032
6,2010-07-08,16.139999,17.520000,15.570000,17.459999,17.459999,7711400,1.950000,11.168386
7,2010-07-09,17.580000,17.900000,16.549999,17.400000,17.400000,4050600,1.350001,7.758626
8,2010-07-12,17.950001,18.070000,17.000000,17.049999,17.049999,2202500,1.070000,6.275660
9,2010-07-13,17.389999,18.639999,16.900000,18.139999,18.139999,2680100,1.739999,9.592057
